In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, Sequential

In [2]:
tf.random.set_seed(2345)

In [3]:
conv_layers = [
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    
    layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    
    layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same')
]

In [4]:
conv_net = Sequential(conv_layers)

In [5]:
conv_net.build(input_shape=[None, 32, 32, 3])

In [6]:
#测试

In [7]:
x = tf.random.normal([4, 32, 32, 3])
out = conv_net(x)

In [8]:
out.shape

TensorShape([4, 1, 1, 512])

In [9]:
###

In [10]:
# 全连接层
fc_net = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(100, activation=None)
])

In [11]:
fc_net.build(input_shape=[None, 512])

In [12]:
optimizer = optimizers.Adam(lr=1e-4)

In [13]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [14]:
(x, y), (x_test, y_test) = datasets.cifar100.load_data()

In [15]:
x.shape, y.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [16]:
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)

In [17]:
train_db = tf.data.Dataset.from_tensor_slices((x,y))
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [18]:
train_db = train_db.shuffle(1000).map(preprocess).batch(64)
test_db = test_db.map(preprocess).batch(64)

In [19]:
sample = next(iter(train_db))
sample[0].shape, sample[1].shape

(TensorShape([64, 32, 32, 3]), TensorShape([64]))

In [ ]:
#
variables = conv_net.trainable_variables + fc_net.trainable_variables

for epoch in range(50):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out = conv_net(x)                        # [b, 32, 32, 3] => [b, 1, 1, 512]
            out = tf.reshape(out, [-1, 512])         # [b, 1, 1, 512] => [b, 512]
            logits = fc_net(out)                     # [b, 512] => [b, 100]
            
            y_onehot = tf.one_hot(y, depth=100)
            
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        
        if step % 100 == 0:
            print(step, 'loss:', float(loss))
    
    total_num = 0
    total_correct = 0
    for x, y in test_db:
        out = conv_net(x)
        out = tf.reshape(out, [-1, 512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits, axis=1)
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        
        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)
        
        total_num += x.shape[0]
        total_correct += int(correct)
    
    acc = total_correct / total_num
    print(epoch, 'acc:', acc)
        